Поиск белых адресов onion-сайтов через заголовок [Onion-Location](https://community.torproject.org/onion-services/advanced/onion-location/) в [Shodan](https://www.youtube.com/watch?v=IOblaXyY2U0&ab_channel=OSINTDojo)

In [13]:
import asyncio
import nest_asyncio

nest_asyncio.apply() 

In [14]:
from telethon import TelegramClient, events, sync

- [Документация](https://github.com/LonamiWebs/Telethon)
- [Конфигурация](https://my.telegram.org/apps)

In [48]:
from urllib.parse import urlparse

def markdown(text) -> list:
    onion_links = []
    # [link text](link)
    for right_part in text.split('](')[1:]:
        link = right_part.split(')')[0]
        if '.onion' in link:
            onion_links.append(urlparse(link).netloc + '\n')
    return onion_links

In [35]:
api_id = None
api_hash = None

async def parse_channel(name, extract_link, debug=False, debug_posts=10):
    async with TelegramClient('onionlinks', api_id, api_hash) as client:
        channel = await client.get_entity(name)
        if debug: 
            posts = 0
        links = []
        async for message in client.iter_messages(channel):
            if message.text:
                links.extend(extract_link(message.text))
            if debug:
                posts += 1
                if posts > debug_posts: 
                    break
        return list(set(links))

In [49]:
def brackets(text) -> list:
    onion_links = []
    for part in text.split(']'):
        if part.startswith('['):
            link = part.lstrip('[')
            if '.onion' in link:
                onion_links.append(urlparse(f'http://{link}').netloc  + '\n')
    return onion_links

In [50]:
def dump_lst(filename, mode, channel):
    with open(filename, mode) as f:
        f.writelines(channel)

In [51]:
dump_lst('onion_links', 'w', await parse_channel('onionnetstalk', brackets))
dump_lst('onion_links', 'a', await parse_channel('onionlinks', markdown))

In [45]:
onionnetstalk = await parse_channel('onionnetstalk', brackets)
with open('onion_links', 'w') as f:
    f.writelines(onionnetstalk)

In [46]:
onionlinks = await parse_channel('onionlinks', markdown)
with open('onion_links', 'a') as f:
    f.writelines(onionlinks)

```
shodan init $APIKEY
for i in `cat onion_links`;do 
  echo "$i"; shodan search $i > $i.txt; sleep 2; 
done
find -name  \*.txt -empty -delete
```